In [11]:
%load_ext blackcellmagic

The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


In [12]:
import datetime
import spiceypy
import numpy as np
import pandas as pd

spiceypy.furnsh("meta.txt")

INIT_TIME_UTC_STR = datetime.datetime(year=2022, month=1, day=1).strftime(
    "%Y-%m-%dT%H:%M:%S"
)
END_TIME_UTC_STR = datetime.datetime(year=2022, month=6, day=1).strftime(
    "%Y-%m-%dT%H:%M:%S"
)

INIT_TIME_ET = spiceypy.utc2et(INIT_TIME_UTC_STR)
END_TIME_ET = spiceypy.utc2et(END_TIME_UTC_STR)

DELTA_HOUR_IN_SECONDS = 3600
TIME_INTERVAL_ET = np.arange(INIT_TIME_ET, END_TIME_ET, DELTA_HOUR_IN_SECONDS)

In [13]:
INNER_SOLSYS_DF = pd.DataFrame()
INNER_SOLSYS_DF.loc[:, "ET"] = TIME_INTERVAL_ET
INNER_SOLSYS_DF.loc[:, "UTC"] = INNER_SOLSYS_DF["ET"].apply(
    lambda x: spiceypy.et2datetime(et=x)
)

INNER_SOLSYS_DF.loc[:, "EARTH_VEN2SUN_ANGLE"] = INNER_SOLSYS_DF["ET"].apply(
    lambda x: np.degrees(
        spiceypy.phaseq(et=x, target="399", illmn="10", obsrvr="299", abcorr="LT+S")
    )
)

INNER_SOLSYS_DF.loc[:, "EARTH_MOON2SUN_ANGLE"] = INNER_SOLSYS_DF["ET"].apply(
    lambda x: np.degrees(
        spiceypy.phaseq(et=x, target="399", illmn="10", obsrvr="301", abcorr="LT+S")
    )
)

INNER_SOLSYS_DF.loc[:, "EARTH_MOON2VEN_ANGLE"] = INNER_SOLSYS_DF["ET"].apply(
    lambda x: np.degrees(
        spiceypy.phaseq(et=x, target="399", illmn="299", obsrvr="301", abcorr="LT+S")
    )
)

In [14]:
INNER_SOLSYS_DF.loc[:, "GOOD_VISIBILITY"] = INNER_SOLSYS_DF.apply(
    lambda x: 1
    if (x["EARTH_VEN2SUN_ANGLE"] > 30.0)
    & (x["EARTH_MOON2SUN_ANGLE"] > 30.0)
    & (x["EARTH_MOON2VEN_ANGLE"] < 10.0)
    else 0,
    axis=1,
)

INNER_SOLSYS_DF

,ET,UTC,EARTH_VEN2SUN_ANGLE,EARTH_MOON2SUN_ANGLE,EARTH_MOON2VEN_ANGLE,GOOD_VISIBILITY
0,6.942673e+08,2022-01-01 00:00:00+00:00,13.084034,25.079659,38.031508,0
1,6.942709e+08,2022-01-01 00:59:59.999999+00:00,13.025223,24.498022,37.395046,0
2,6.942745e+08,2022-01-01 01:59:59.999998+00:00,12.966408,23.916378,36.758716,0
3,6.942781e+08,2022-01-01 02:59:59.999996+00:00,12.907590,23.334757,36.122553,0
4,6.942817e+08,2022-01-01 03:59:59.999995+00:00,12.848770,22.753195,35.486595,0
...,...,...,...,...,...,...
3620,7.072993e+08,2022-05-31 19:59:59.999007+00:00,36.619036,15.068923,51.594252,0
3621,7.073029e+08,2022-05-31 20:59:59.999008+00:00,36.610102,15.521491,52.040183,0
3622,7.073065e+08,2022-05-31 21:59:59.999009+00:00,36.601166,15.974072,52.485999,0
3623,7.073101e+08,2022-05-31 22:59:59.999010+00:00,36.592227,16.426655,52.931705,0


In [15]:
print(
    f"Good visibility will be available for {len(INNER_SOLSYS_DF.loc[INNER_SOLSYS_DF['GOOD_VISIBILITY'] == 1])} hours in this time period"
)

Good visibility will be available for 122 hours in this time period
